In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

#图的保存
v = tf.Variable(1.0,name='my_variable')
with tf.Session() as sess:
    tf.train.write_graph(sess.graph_def,'./tfmodel','test_pb.pb',as_text=False)
    


In [ ]:
#图的加载
with tf.Session() as sess:
    with tf.gfile.FastGFile('./tfmodel/test_pb.pb','rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        tf.import_graph_def(graph_def,name='tf.graph')
        print(graph_def)
    

In [1]:
#导入inception模型
from __future__ import print_function
import os
from io import BytesIO
import numpy as np 
from functools import partial
import PIL.Image
import scipy.misc
import tensorflow as tf



In [3]:

graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
model_fn = 'tensorflow_inception_graph.pb'#导入inception模型
with tf.gfile.GFile(model_fn,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
#定义输入图像的占位符
t_input = tf.placeholder(np.float32,name='input')
#图像预处理 减均值 在训练inception模型时做了减均值处理 在此也需减同样的均值以保持一致

imagenet_mean = 117.0

#图像预处理 增加维度
#图像数据格式就是 （hight width channels）为同时将多张图片输入网络而在掐面增加一维
#变成（ batch hight width channels）

t_preprocessed = tf.expand_dims(t_input - imagenet_mean,0)
#导入模型并将预处理的图像送入网络中
tf.import_graph_def(graph_def,{'input':t_preprocessed})

E:\soft\anaconda1\envs\tensor1\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [5]:
layers = [op.name for op in graph.get_operations() if op.type =='Conv2D' and 'import/' in op.name]
print(layers)
print('NUMber of layers' , len(layers))

['import/conv2d0_pre_relu/conv', 'import/conv2d1_pre_relu/conv', 'import/conv2d2_pre_relu/conv', 'import/mixed3a_1x1_pre_relu/conv', 'import/mixed3a_3x3_bottleneck_pre_relu/conv', 'import/mixed3a_3x3_pre_relu/conv', 'import/mixed3a_5x5_bottleneck_pre_relu/conv', 'import/mixed3a_5x5_pre_relu/conv', 'import/mixed3a_pool_reduce_pre_relu/conv', 'import/mixed3b_1x1_pre_relu/conv', 'import/mixed3b_3x3_bottleneck_pre_relu/conv', 'import/mixed3b_3x3_pre_relu/conv', 'import/mixed3b_5x5_bottleneck_pre_relu/conv', 'import/mixed3b_5x5_pre_relu/conv', 'import/mixed3b_pool_reduce_pre_relu/conv', 'import/mixed4a_1x1_pre_relu/conv', 'import/mixed4a_3x3_bottleneck_pre_relu/conv', 'import/mixed4a_3x3_pre_relu/conv', 'import/mixed4a_5x5_bottleneck_pre_relu/conv', 'import/mixed4a_5x5_pre_relu/conv', 'import/mixed4a_pool_reduce_pre_relu/conv', 'import/mixed4b_1x1_pre_relu/conv', 'import/mixed4b_3x3_bottleneck_pre_relu/conv', 'import/mixed4b_3x3_pre_relu/conv', 'import/mixed4b_5x5_bottleneck_pre_relu/conv',

In [9]:
#还可指出指定卷积层的参数
name1 = 'mixed4d_3x3_bottleneck_pre_relu'
print('shape of %s:%s'%(name1,str(graph.get_tensor_by_name('import/'+name1+':0').get_shape())))

name2 = 'mixed4d_5x5_bottleneck_pre_relu'
print('shape of %s:%s'%(name2,str(graph.get_tensor_by_name('import/'+name2+':0').get_shape())))

shape of mixed4d_3x3_bottleneck_pre_relu:(?, ?, ?, 144)
shape of mixed4d_5x5_bottleneck_pre_relu:(?, ?, ?, 32)


In [11]:
import numpy as np 
import tensorflow as tf 

g = tf.Graph()
with g.as_default():
    c1 = tf.constant(0.0)
    print('c1.graph:',c1.graph)

c1.graph: <tensorflow.python.framework.ops.Graph object at 0x00000214D34E5A48>


In [ ]:
#渲染函数
def render_naive(t_obj,img0,iter_n=20,step=1.0):
    #t_obj是layer_output[:,:,:,channel],即卷积层某个通道的值
    #img0 初始图像（噪声图像）
    #iter_n 迭代次数
    # step:用于控制迭代步长 可以看做学习率
    
    t_score = tf.reduce_mean(t_obj)
    #t_c=score是t_obj的平均值
    #由于我们的目标是调整输入图像使卷积层激活值尽量大
    #即最大化t_score
    #为达到此目标 可使用梯度下降
    #计算t_score对t_input的梯度
    t_grad = tf.gradients(t_score,t_input)[0]
    img = img0.copy()#复制图像可避免影响原图像的值
    for i in range(iter_n):
        #在sess中计算梯度以及当前的t_score
        g,score = sess.run([t_grad,t_score],{t_input:img})
        #对img应用梯度
        #首先对梯度进行归一化处理
        g /= g.std() + le-8
        #将正规化处理后的梯度应用在图像上 step 用于控制每次迭代步长 此处为1.0
        img += g*step
        #print('score(mean)=%f'%(score))
        print('iter:%d'%(i+1),'score(mean)=%f'%score)
        
        #保存图片
    savearray(img,'naive_deepddream.jpg')








#定义卷积层 通道数 并取出对应的tensor
name = 'mixed4d_3x3_bottleneck_pre_relu'
#(?,?,?,144)
#name = 'mixed4d_5x5_bottleneck_pre_relu'
#(?,?,?,32)
channel = 139
# mixed4d_3x3_bottleneck_pre_relu 共144个通道
#此处可选任意通道（0-143）之间的任意整数 进行最大化
layer_output = graph.get_tensor_by_name('import/%s:0'%name)
# 定义噪声图像
img_noise = np.random.uniform(size=(224,224,3))+100.0

#调用render_naive函数渲染
render_naive(layer_output[:,:,:,channel],img_noise,iter_n=20)
# 保存并显示图片
im = PIL.Image.open('mountain.jpg')
im.show()
im.save('naive_single_chn.jpg')